## What are the SOLID principles?

### Single Responsibility Principle (SRP)
The Single Responsibility Principle advocates for a class or module to have only one reason to change. In simpler terms, it should do one thing and do it well. By adhering to SRP, your code becomes more modular, making it easier to understand and maintain.

In [3]:
class Order:
    def __init__(self) -> None:
        self.items = []
        self.quantites = []
        self.prices = []
        self.status = 'open'
    
    def add_item(self,name:str,quantity:int,price:float)->None:
        self.items.append(name)
        self.quantites.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for quantity,price in zip(self.quantites,self.prices):
            total += (quantity*price)
        return total
    
    def pay(self,payment_type:str,security_code):
        if payment_type == 'debit':
            print("processing debit type payment")
            print(f"verfiying security code: {security_code}")
            self.status = "Paid"
        elif payment_type == 'credit':
            print("processing credit type payment")
            print(f"verfiying security code: {security_code}")
            self.status = "Paid"
        else:
            raise Exception(f"unknown payment type : {payment_type}")

order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)
print(order.total_price())
order.pay("debit", "0372846")
        

210
processing debit type payment
verfiying security code: 0372846


so above program voilate the SRP because its is both responsible for managing order and payment.
above code is highly coupled and harder to understand and manage.

In [7]:
class Order:
    def __init__(self) -> None:
        self.items = []
        self.quantites = []
        self.prices = []
        self.status = 'open'
    
    def add_item(self,name:str,quantity:int,price:float)->None:
        self.items.append(name)
        self.quantites.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for quantity,price in zip(self.quantites,self.prices):
            total += (quantity*price)
        return total

class PaymentProcessor:
    def pay(self,order:Order,security_code):
            print("processing payment")
            print(f"verfiying security code: {security_code}")
            order.status = "Paid"


order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)
print(order.total_price())
payment = PaymentProcessor()
payment.pay(order,'200202')
print(order.status)

210
processing payment
verfiying security code: 200202
Paid


### Open-Closed Principle(OCP)
The Open-Closed Principle states that software entities should be open for extension but closed for modification. 
##### This means that you should be able to extend a class’s behavior without modifying it.

so in above we unable to add new payment method like I want to add paypal but unable to add that so it violate the OCP because it should be open for extension but closed for modification.

In [9]:
#OSP
#order class will same

from abc import ABC,abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self,order,security_code:str):
        ...

class CreditCardPaymentProcessor(PaymentProcessor):
    def pay(self,order,security_code:str):
        print("Processing credit card payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"



class DebitCardPaymentProcessor(PaymentProcessor):
    def pay(self,order,security_code:str):
        print("Processing debit card payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class PayPalCardPaymentProcessor(PaymentProcessor):
    def pay(self,order,security_code:str):
        print("Processing payPal payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = PayPalCardPaymentProcessor()
processor.pay(order,"044004")
print(order.status)

# Now that the code adheres to the OCP, it’s closed for modification and open for
# extension because we can add new payment methods without modifying the PaymentProcessor class.

SyntaxError: invalid character '’' (U+2019) (3805455618.py, line 43)

### Liskov Substitution Principle (LSP)
The Liskov Substitution Principle states that objects in a program should be replaceable with instances of their subtypes without altering the correctness of the program. 
##### In other words, a subclass should be able to replace its parent class without breaking the code.

so above refectoring , we violate LSP because paypal use email address and other user security card so means subclass is not compatible with its parent class

In [12]:
#LSP
#OSP
#order class will same

from abc import ABC,abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self,order):
        ...

class CreditCardPaymentProcessor(PaymentProcessor):
    def __init__(self,security_code:str) -> None:
        self.security_card = security_code
    def pay(self,order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"



class DebitCardPaymentProcessor(PaymentProcessor):
    def __init__(self,security_code:str) -> None:
        self.security_card = security_code
    def pay(self,order):
        print("Processing debit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PayPalCardPaymentProcessor(PaymentProcessor):
    def __init__(self,email:str) -> None:
        self.email = email

    def pay(self,order):
        print("Processing payPal payment")
        print(f"Verifying security code: {self.email}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = PayPalCardPaymentProcessor("abc@gmail.com")
processor.pay(order)
print(order.status)

# Now the code adheres to the LSP because we are
# using the subclasses in a way that is compatible with their parent class.

Processing payPal payment
Verifying security code: abc@gmail.com
paid


### Interface Segregation Principle (ISP)
The Interface Segregation Principle states that clients should not be forced to depend on methods they do not use.
##### This means that you should not have to implement methods that you do not need.

so in below credit card not need sms authentication but we have to implement this so it violated ISP for this different class

In [30]:
#ISP
#LSP
#OSP
#order class will same

from abc import ABC,abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self,order):
        ...
    
    # @abstractmethod
    # def auth_sms(self,order,code:str): #this method need to move
    #     ...

class SMSProcessor(PaymentProcessor): #so its satisfy the LSP rule this should be pass when we need auth
    @abstractmethod
    def auth_sms(self,order,code:str): 
        ...

#but we have to move authrization logic into different class

class SMSAuthrizer:
    def __init__(self) -> None:
        self.is_auth = False
    
    def verfiy_code(self,code:str):
        print("verify the code under sms class")
        self.is_auth = True
    
    def is_authenticated(self):
        return self.is_auth

class CreditCardPaymentProcessor(PaymentProcessor):
    def __init__(self,security_code:str) -> None:
        self.security_code = security_code
        # self.authenticated = False
    def pay(self,order):
        # if not self.authenticated:
        #     raise Exception("Not Authenticated")
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"
    
    # def auth_sms(self, order, code: str):
    #     print("Authenticating via SMS")
    #     self.authenticated = True

class PayPalCardPaymentProcessor(PaymentProcessor):
    def __init__(self,email:str,authorizer:SMSAuthrizer) -> None:
        self.email = email
        self.authrizer = authorizer

    def pay(self,order):
        if not self.authrizer.is_authenticated():
            raise Exception("Not Authenticated")
        print("Processing payPal payment")
        print(f"Verifying security code: {self.email}")
        order.status = "paid"
    
    # def auth_sms(self, order, code: str): #no need to use when we are seperated sms authrization
    #     print("Authenticating via SMS")
    #     self.authenticated = True


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)
sms = SMSAuthrizer()
sms.verfiy_code("04004")
processor = PayPalCardPaymentProcessor("abc@gmail.com",sms)
# processor.auth_sms(order,"abc@gmail.com")
processor.pay(order)
print(order.status)
print("--------------------")
processor_credit = CreditCardPaymentProcessor("04004")
# processor.auth_sms(order,"404040")
processor_credit.pay(order)
print(order.status)

# Now the code adheres to the LSP because we are
# using the subclasses in a way that is compatible with their parent class.

verify the code under sms class
Processing payPal payment
Verifying security code: abc@gmail.com
paid
--------------------
Processing credit card payment
Verifying security code: 04004
paid


### Dependency Inversion Principle (DIP)
The Dependency Inversion Principle states that high-level modules should not depend on low-level modules, but both should depend on abstractions. 
##### This means that you should not have to change your code when you change the implementation of a module.

In [33]:
#DIP
#order class same


from abc import ABC,abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self,order):
        ...


class Authrizer(ABC):
    def is_authenticated(self):
        ...
    

class SMSAuthrizer(Authrizer):
    def __init__(self) -> None:
        self.is_auth = False
    
    def verfiy_code(self):
        print("verify the code under sms class")
        self.is_auth = True
    
    def is_authenticated(self):
        return self.is_auth

class NotARobotAuthrizer(Authrizer):
    def __init__(self) -> None:
        self.is_auth = False
    
    def ask(self):
        print("Are you a robot?!!! [┐∵]┘")
        self.is_auth = True
    
    def is_authenticated(self):
        return self.is_auth


class DebitCardPaymentProcessor(PaymentProcessor):
    def __init__(self,security_code:str,authrizer:Authrizer) -> None:
        self.security_card = security_code
        self.authrizer = authrizer
    def pay(self,order):
        if self.authrizer.is_authenticated():
            raise Exception("Not authenticated")
        
        print("Processing debit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditCardPaymentProcessor(PaymentProcessor):
    def __init__(self,security_code:str) -> None:
        self.security_code = security_code
    def pay(self,order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"
    

class PayPalCardPaymentProcessor(PaymentProcessor):
    def __init__(self,email:str,authorizer:Authrizer) -> None:
        self.email = email
        self.authrizer = authorizer

    def pay(self,order):
        if not self.authrizer.is_authenticated():
            raise Exception("Not Authenticated")
        print("Processing payPal payment")
        print(f"Verifying security code: {self.email}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor_credit = CreditCardPaymentProcessor("04004")
processor_credit.pay(order)
print(order.status)
print("________________")
sms = SMSAuthrizer()
sms.verfiy_code()
processor = PayPalCardPaymentProcessor("abc@gmail.com",sms)
processor.pay(order)
print(order.status)
print("--------------------")
robot = NotARobotAuthrizer()
robot.ask()
processor_debit = DebitCardPaymentProcessor("43000304",robot)
processor.pay(order)
print(order.status)

# Now the code adheres to the LSP because we are
# using the subclasses in a way that is compatible with their parent class.

Processing credit card payment
Verifying security code: 04004
paid
________________
verify the code under sms class
Processing payPal payment
Verifying security code: abc@gmail.com
paid
--------------------
Are you a robot?!!! [┐∵]┘
Processing payPal payment
Verifying security code: abc@gmail.com
paid


Thanks to https://www.arjancodes.com/blog/solid-principles-in-python-programming/